In [ ]:
import os, sys, random

import pandas as pd
import numpy as np
import swifter

from datetime import timedelta, date
from importlib import reload
from data_warehouse_utils.dataloader import DataLoader

from causal_inference.experiment_generator.create_observations import create_observations
from causal_inference.experiment_generator.create_covariates import add_covariates

In [ ]:
# Reloads packages
os.chdir('/home/adam/files/causal_inference')
os.getcwd()

reload(sys.modules['causal_inference.create_experiment.create_treatment'])
reload(sys.modules['causal_inference.create_experiment.create_observations'])
reload(sys.modules['causal_inference.create_experiment.create_covariates'])

from causal_inference.create_experiment.create_observations import create_observations
from causal_inference.create_experiment.create_covariates import add_covariates

In [ ]:
dl = DataLoader()

In [ ]:
df = create_observations(dl)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.hash_patient_id.nunique()

In [ ]:
df_sample = df.sample(5)
df_sample.info()

In [ ]:
df_covariates, df_timestamp = add_covariates(dl=dl,
                                             df=df,
                                             interval_start=12,
                                             interval_end=0)

In [ ]:
df_covariates.info()

In [ ]:
df_timestamp.describe()

In [ ]:
os.chdir('/home/adam/files/data')
os.getcwd()

df_covariates.to_csv('data_skeleton.csv', index=False)

In [ ]:
df_dropped = df_covariates.dropna()

In [ ]:
df_dropped[df_dropped.treated == True].info()


In [ ]:
df_treatment_new[df_treatment_new.treated == True].dropna().info()

In [ ]:
df_treatment_new[df_treatment_new.treated == True].info()

In [ ]:
df[(df.treated == False) & (df.duration_hours > 12 )].describe()



In [ ]:
def delete_shorter(df, df_time):

In [ ]:
df_covariates.head()

In [ ]:
df_timestamp.head()

In [ ]:
for idx, row in df_timestamp.iterrows():
    if df_timestamp[idx] is in something then np.Nan the same
    